In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import lightgbm as lgb 
import catboost as cb 


In [2]:
SEED=42

In [85]:
train=pd.read_csv('data/final.csv')
test=pd.read_csv('data/test.csv')

In [86]:
train.head()

,ID,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,29,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1
1,57,48,1,0,122,222,0,0,186,0,0.0,2,0,2,1
2,9,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
3,147,44,0,2,118,242,0,1,149,0,0.3,1,1,2,1
4,303,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


In [87]:
train.target.value_counts()

1    137
0    116
Name: target, dtype: int64

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        253 non-null    int64  
 1   age       253 non-null    int64  
 2   sex       253 non-null    int64  
 3   cp        253 non-null    int64  
 4   trestbps  253 non-null    int64  
 5   chol      253 non-null    int64  
 6   fbs       253 non-null    int64  
 7   restecg   253 non-null    int64  
 8   thalach   253 non-null    int64  
 9   exang     253 non-null    int64  
 10  oldpeak   253 non-null    float64
 11  slope     253 non-null    int64  
 12  ca        253 non-null    int64  
 13  thal      253 non-null    int64  
 14  target    253 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 29.8 KB


In [6]:
data=train.append(test)
data.drop(['ID'],axis=1)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1.0
1,48,1,0,122,222,0,0,186,0,0.0,2,0,2,1.0
2,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1.0
3,44,0,2,118,242,0,1,149,0,0.3,1,1,2,1.0
4,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,70,1,1,156,245,0,0,143,0,0.0,2,0,2,NaN
46,61,1,2,150,243,1,1,137,1,1.0,1,0,2,NaN
47,64,1,0,128,263,0,1,105,1,0.2,1,1,3,NaN
48,54,1,0,124,266,0,0,109,1,2.2,1,1,3,NaN


In [7]:
train=data[0:train.shape[0]]
test=data[train.shape[0]:]
print(train.shape,test.shape)

(253, 15) (50, 15)


In [8]:
y=train.target
x=train.drop(['target'],axis=1)

In [9]:
test=test.drop(['target'],axis=1)

In [10]:
skf=StratifiedKFold(n_splits=3,random_state=SEED)

In [11]:
scores=[]
fold=0
for train_idx,test_idx in skf.split(x,y):
    x_train,x_test=x.iloc[train_idx],x.iloc[test_idx]
    y_train,y_test=y.iloc[train_idx],y.iloc[test_idx]
    print(fold)
    fold+=1

    lg=lgb.LGBMClassifier(random_state=SEED,n_estimators=2000,loss='rmse')
    lg.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=150,verbose=50)
    preds=lg.predict(x_test)
    score=f1_score(y_test,preds,average=None)
    print(score)
    scores.append(score)
print(np.mean(scores),np.std(scores))

0
[LightGBM] [Warning] Unknown parameter: loss
Training until validation scores don't improve for 150 rounds
[50]	valid_0's binary_logloss: 0.00592809
[100]	valid_0's binary_logloss: 0.000158071
[150]	valid_0's binary_logloss: 3.81477e-05
[200]	valid_0's binary_logloss: 3.81477e-05
[250]	valid_0's binary_logloss: 3.81477e-05
Early stopping, best iteration is:
[140]	valid_0's binary_logloss: 3.79695e-05
[1. 1.]
1
[LightGBM] [Warning] Unknown parameter: loss
Training until validation scores don't improve for 150 rounds
[50]	valid_0's binary_logloss: 0.00481278
[100]	valid_0's binary_logloss: 6.44545e-05
[150]	valid_0's binary_logloss: 1.91833e-05
[200]	valid_0's binary_logloss: 1.89234e-05
[250]	valid_0's binary_logloss: 1.89231e-05
[300]	valid_0's binary_logloss: 1.89231e-05
Early stopping, best iteration is:
[178]	valid_0's binary_logloss: 1.89144e-05
[1. 1.]
2
[LightGBM] [Warning] Unknown parameter: loss
Training until validation scores don't improve for 150 rounds
[50]	valid_0's bina

In [12]:
scores=[]
fold=0
for train_idx,test_idx in skf.split(x,y):
    x_train,x_test=x.iloc[train_idx],x.iloc[test_idx]
    y_train,y_test=y.iloc[train_idx],y.iloc[test_idx]
    print(fold)
    fold+=1

    cat=cb.CatBoostClassifier(random_state=SEED,n_estimators=5000)
    cat.fit(x_train,y_train,eval_set=[(x_test,y_test)],early_stopping_rounds=150,verbose=100,use_best_model=True)
    preds=cat.predict(x_test)
    score=f1_score(y_test,preds,average=None)
    print(score)
    scores.append(score)
print(np.mean(scores),np.std(scores))

0
Learning rate set to 0.010128
0:	learn: 0.6757553	test: 0.6782042	best: 0.6782042 (0)	total: 149ms	remaining: 12m 24s
100:	learn: 0.0849740	test: 0.0959386	best: 0.0959386 (100)	total: 239ms	remaining: 11.6s
200:	learn: 0.0293063	test: 0.0396400	best: 0.0396400 (200)	total: 335ms	remaining: 7.99s
300:	learn: 0.0149161	test: 0.0242644	best: 0.0242644 (300)	total: 431ms	remaining: 6.72s
400:	learn: 0.0095137	test: 0.0186466	best: 0.0186466 (400)	total: 522ms	remaining: 5.99s
500:	learn: 0.0067506	test: 0.0161966	best: 0.0161966 (500)	total: 627ms	remaining: 5.63s
600:	learn: 0.0051002	test: 0.0147556	best: 0.0147556 (600)	total: 725ms	remaining: 5.3s
700:	learn: 0.0042198	test: 0.0140809	best: 0.0140809 (700)	total: 823ms	remaining: 5.05s
800:	learn: 0.0036011	test: 0.0135761	best: 0.0135761 (800)	total: 923ms	remaining: 4.84s
900:	learn: 0.0030704	test: 0.0131113	best: 0.0131113 (900)	total: 1.02s	remaining: 4.64s
1000:	learn: 0.0026464	test: 0.0126414	best: 0.0126414 (1000)	total: 1.

In [13]:
from sklearn.linear_model import LogisticRegression
scores=[]
fold=0
for train_idx,test_idx in skf.split(x,y):
    x_train,x_test=x.iloc[train_idx],x.iloc[test_idx]
    y_train,y_test=y.iloc[train_idx],y.iloc[test_idx]
    print(fold)
    fold+=1

    lr=LogisticRegression()
    lr.fit(x_train,y_train)
    preds=lr.predict(x_test)
    score=f1_score(y_test,preds,average=None)
    print(score)
    scores.append(score)
print(np.mean(scores),np.std(scores))

0
[0.95       0.95555556]
1
[1. 1.]
2
[0.96       0.96774194]
0.9722162485065712 0.020348096905322462


In [14]:
lg.feature_importances_

array([145,  74,   0,  69,  39,  45,   0,  26,  83,   5,  25,   0,   2,
         0])

In [76]:
test_preds=cat.predict(test)

In [77]:
test_preds

array([1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.])

In [78]:
sub=pd.read_csv('data/test.csv')

In [79]:
sub.head()

,ID,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,128,67,0,2,152,277,0,1,172,0,0.0,2,1,2
1,232,57,1,0,165,289,1,0,124,0,1.0,1,3,3
2,220,48,1,0,130,256,1,0,150,1,0.0,2,2,3
3,184,58,1,2,112,230,0,0,165,0,2.5,1,1,3
4,25,40,1,3,140,199,0,1,178,1,1.4,2,0,3


In [80]:
new_df=pd.DataFrame({'ID':sub.ID,'target':test_preds})

In [81]:
new_df.head()

,ID,target
0,128,1.0
1,232,0.0
2,220,0.0
3,184,0.0
4,25,1.0


In [82]:
new_df.target.value_counts()

1.0    28
0.0    22
Name: target, dtype: int64

In [83]:
new_df.to_csv('submissions/initial-cat-0.9920.csv',index=False)

In [84]:
new_df

,ID,target
0,128,1.0
1,232,0.0
2,220,0.0
3,184,0.0
4,25,1.0
5,56,1.0
6,28,1.0
7,125,1.0
8,139,1.0
9,67,1.0
